
The aim of the exercise is to get familiar with the histogram gradient-boosting in scikit-learn. Besides, we will use this model within a cross-validation framework in order to inspect internal parameters found via grid-search.

We will use the California housing dataset.

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load dataset
data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100  # Rescale target to k$

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)


### 📝 TODO: First, create a histogram gradient boosting regressor. You can set the trees number to be large, and configure the model to use early-stopping.

In [7]:
random_state=42

# Define the Histogram Gradient Boosting Regressor
hgb_regressor = HistGradientBoostingRegressor(
    max_iter=1000,  # Large number of trees
    early_stopping=True,  # Enable early stopping
    scoring="neg_mean_squared_error",
    validation_fraction=0.1,
)

# Fit the model
hgb_regressor.fit(X_train, y_train)

# Print model performance
print("Training Score:", hgb_regressor.score(X_train, y_train))
print("Test Score:", hgb_regressor.score(X_test, y_test))

Training Score: 0.9177551717531773
Test Score: 0.8482980797850956


### 📝 TODO: Use `RandomizedSearchCV` with `n_iter=20` to find the best set of
hyperparameters by tuning the following parameters of the `model`:
- max_depth: [3, 8];
- max_leaf_nodes: [15, 31];
- learning_rate: [0.1, 1].

Notice that in the notebook "Hyperparameter tuning by randomized-search" we
pass distributions to be sampled by the `RandomizedSearchCV`. In this case we
define a fixed grid of hyperparameters to be explored. Using a `GridSearchCV`
instead would explore all the possible combinations on the grid, which can be
costly to compute for large grids, whereas the parameter `n_iter` of the
`RandomizedSearchCV` controls the number of different random combination that
are evaluated. Notice that setting `n_iter` larger than the number of possible
combinations in a grid would lead to repeating
already-explored combinations. Here, this can't happen since the learning rate is sampled from a uniform so the number of possible combinations is infinite.

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import numpy as np

# Define parameter grid
param_distributions = {
    "max_depth": [3, 8],
    "max_leaf_nodes": [15, 31],
    "learning_rate": [0.1, 1]
}

# Initialize the model
hgb_regressor = HistGradientBoostingRegressor(
    max_iter=1000,  # Keep a large number of trees
    early_stopping=True,
    scoring="neg_mean_squared_error",
    validation_fraction=0.1,
    random_state=42
)

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=hgb_regressor,
    param_distributions=param_distributions,
    n_iter=20,
    scoring="neg_mean_squared_error",
    cv=5,  # 5-fold cross-validation
    random_state=42,
    n_jobs=-1  # Use all available CPUs
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Hyperparameters: {'max_leaf_nodes': 31, 'max_depth': 8, 'learning_rate': 0.1}
Best Score: -2159.1499843772317


📝 TODO: Secondly, we will run our experiment through cross-validation. In this regard, define a 5-fold cross-validation. Besides, be sure to shuffle the data. Subsequently, use the function sklearn.model_selection.cross_validate to run the cross-validation. You should also set return_estimator=True, so that we can investigate the inner model trained via cross-validation.

In [ ]:
from sklearn.model_selection import cross_validate, KFold
# CODE HERE
cv =
cross_val_results =

### 📝 TODO: Now that we got the cross-validation results, print out the mean and standard deviation score.

In [ ]:
cv_mean, cv_sd =

### 📝 TODO: Then inspect the estimator entry of the results and check the best parameters values. Besides, check the number of trees used by the model.

In [ ]:
# CODE HERE
for estimator in :

### 📝 TODO: Inspect the results of the inner CV for each estimator of the outer CV. Aggregate the mean test score for each parameter combination and make a box plot of these scores.

In [ ]:
import pandas as pd
# CODE HERE
inner_cv_results = []
for cv_ix, estimator_ in :
    results =

inner_cv_results = pd.concat(inner_cv_results, axis=1).T

In [ ]:
import matplotlib.pyplot as plt

# Sort the inner_cv_results by the mean of the columns
sorted_inner_cv_results = inner_cv_results.mean().sort_values().index
inner_cv_results = inner_cv_results[sorted_inner_cv_results]

color = {"whiskers": "black", "medians": "black", "caps": "black"}
inner_cv_results.plot.box(vert=False, color=color)
plt.xlabel("R2 score")
plt.ylabel("Parameters")
_ = plt.title(
    "Inner CV results with parameters\n" "(max_depth, max_leaf_nodes, learning_rate)"
)

We see that the first 4 ranked set of parameters are very close. We could select any of these 4 combinations. It coincides with the results we observe when inspecting the best parameters of the outer CV.